In [ ]:
using Globtim
using CSV
using DataFrames
include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d = 3      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
# const tol_l2 = 1e-1               # Define the tolerance for the L2-norm
const tol_l2 = 1e-0
# The objective function
f = camel_3_by_3 # Objective function

In [ ]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

In [ ]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d)) # Convert coefficients to Float64 for homotopy continuation
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [ ]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

Construct a Dataframe with only the critical points of the approximant $w_d$ which fall into the $[-1, 1]^4$ domain and rescale them to the original domain. 

In [5]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1 && -1 < point[3] < 1 && -1 < point[4] < 1 
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x1 = scale_factor * Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_x2 = scale_factor * Float64[point[2] for point in filtered_points] 
h_x3 = scale_factor * Float64[point[3] for point in filtered_points] 
h_x4 = scale_factor * Float64[point[4] for point in filtered_points] 

height = map(p -> f(p), zip(h_x1, h_x2, h_x3, h_x4)) # Compute the height of the critical points
df = DataFrame(x1 = h_x1, x2 = h_x2, x3 = h_x3, x4 = h_x4, height = height); # Create a DataFrame

Now we want to compute the distance between the critical points of the 2d function.

The true "exact" critical points of the 2d `Camel3` function are stored in `df_2d`. 

We compute which of those are the closest to the `[1:2]` and `[3:4]` coordinates of the critical points of our 4d approximant.  

The true critical points are stored in `df_2d`. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [ ]:
total_distance = Float64[]
for i in 1:nrow(df)
    distances_1 = [norm([df.x1[i], df.x2[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.x3[i], df.x4[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    # Compute distances to all points in df_2d
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2) 

    # Extract the corresponding points
    point_1 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    point_2 = [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]]

    # Compute total distance
    tot_dist = norm(point_1 - point_2)
    push!(total_distance, tot_dist)

end
df.total_distance = total_distance
sorted_df = sort(df, :total_distance)


In [ ]:
using Optim
df.steps = zeros(nrow(df))
df.converged = zeros(nrow(df))

for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    # df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
df